In [1]:
from ecdsa import SigningKey
from hashlib import sha256
import copy
from random import randint

In [2]:
def splitnumber(str1):
        n=len(str1)//2
        return(str1[:n],str1[n:])
    
class coldwallet:
    def __init__(self,seed):
        self.msk=SigningKey.generate()
        #use seed to generate hash string and get the second part of it as initstate
        self.st=splitnumber(sha256(str(seed).encode("utf-8")).hexdigest())[1]
        self.sessionkeys={}
        
    def generatehotwallet(self):
        return hotwallet(self.verificationmasterkey(),self.st)
    
    def verificationmasterkey(self):
        return self.msk.get_verifying_key()
        
    def createsessionkey(self,ID):
        if ID in self.sessionkeys:
            return False
        pair=splitnumber(sha256((self.st+str(ID)).encode("utf-8")).hexdigest())
        #update state first
        self.st=pair[1]
        #create sessionsecretkey and store it
        sessionprivkey=copy.deepcopy(self.msk.privkey)
        sessionprivkey.secret_multiplier*=int(pair[0],16)
        sessionprivkey.public_key.point*=int(pair[0],16)
        self.sessionkeys[ID]=sessionprivkey
        return True
    
    def sessionverificationkey(self,ID):
        if ID in self.sessionkeys:
            return self.sessionkeys[ID].public_key
        return None
                        
    def sign(self,ID,message):
        hashmsg=int(sha256(message.encode("utf-8")).hexdigest(),16)
        randomnumber=int(sha256(str(hash(randint(10000,10000000))).encode("utf-8")).hexdigest(),16)
        return self.sessionkeys[ID].sign(hashmsg,randomnumber)

class hotwallet:  
    def __init__(self,mpk,st):
        self.mpk=mpk
        self.st=st
        self.sessionpublickeys={}
    
    def createsessionpubkey(self,ID):
        if ID in self.sessionpublickeys:
            return False
        pair=splitnumber(sha256((self.st+str(ID)).encode("utf-8")).hexdigest())
        #update state first
        self.st=pair[1]
        #create sessionpublickey and store it
        sessionpubkey=copy.deepcopy(self.mpk.pubkey)
        sessionpubkey.point*=int(pair[0],16)
        self.sessionpublickeys[ID]=sessionpubkey
        return True
    def verification(self,ID,signature,msg):
        hashmsg=int(sha256(message.encode("utf-8")).hexdigest(),16)
        return self.sessionpublickeys[ID].verifies(hashmsg,signature)



In [3]:
cw=coldwallet(114514)
hw=cw.generatehotwallet()
IDlist=["Microsoft","Amazon","Google","Alibaba"]
message="Donald Trump"
for ID in IDlist:
    cw.createsessionkey(ID)
    hw.createsessionpubkey(ID)

for ID in IDlist:
    signature=cw.sign(ID,message)
    print("Yes, signature is "+str(hw.verification(ID,signature,message)))

Yes, signature is True
Yes, signature is True
Yes, signature is True
Yes, signature is True
